## Core

In [ ]:
# !pip install requests
!pip3 install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.8/232.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.13.0-py3-none-any.whl size=214529 sha256=c476ca9902f5a406f639c45d9a238fbf17bf3c81791364f16f572f39a09a3a7e
  Stored in directory: /root/.cache/pip/wheels/f7/1a/3b/41e1ce64625e3f11567085dc79043d9fa573a2cc39f9938556
Successfully built pytelegrambotapi


In [ ]:
import requests
import json
from time import sleep

import telebot
from telebot import types

## API class

In [ ]:
class CountryLayerApi:
    def __init__(self):
        self.__response = None
        self.__url = "http://api.countrylayer.com/v2/"
        self.__params = {"access_key" : "b02128de0572396cc66a1d8097aa6de6" }                                    # Personal key
        self.__errors_numbers = {404 : "Error 404: The requested resource does not exist.", \
                                 401 : "Error 401: Unauthorized.", \
                                 403 : "Error 403: Forbidden.", \
                                 101 : "Error 101: No API Key was specified or an invalid API Key was specified.", \
                                 103 : "Error 103: The requested API endpoint does not exist.", \
                                 104 : "Error 104: The maximum allowed API amount of monthly API requests has been reached.", \
                                 429 : "Error 429: Too Many Requests."}

    def get_request_to_api(self, url):
        return (self.handdle_errors(response = requests.get(url, params=self.__params)))

    def request_all(self):
        return self.get_request_to_api(self.__url + "all")

    def request_country_name(self, country_name):
        return self.get_request_to_api(self.__url + f"name/{country_name}")

    def request_country_capital(self, capital):
        return self.get_request_to_api(self.__url + f"capital/{capital}")

    def request_country_language(self, language):
        return self.get_request_to_api(self.__url + f"language/{language}")

    def request_country_currency(self, currency):
        return self.get_request_to_api(self.__url + f"currency/{currency}")

    def request_country_region(self, region):
        return self.get_request_to_api(self.__url + f"region/{region}")

    def request_country_regionalbloc(self, regionalbloc):                           # example : "EU/RU/"
        return self.get_request_to_api(self.__url + f"regionalbloc/{regionalbloc}")

    def request_country_calling_code(self, callingcode):
        return self.get_request_to_api(self.__url + f"callingcode/{callingcode}")

    def handdle_errors(self, response):
        if response.status_code == 200:
            return self.response_to_user(response)
        elif response.status_code in self.__errors_numbers:
            return(self.__errors_numbers[response.status_code])
        else:
            return "Request error. Try again."

    def response_to_user(self, response):
        return response

tests


## Telebot

In [ ]:
token = '6244709804:AAE35sTigDut3AmYuwl6y50J0_eE8JBrQQk'                                                                      # Personal token
bot = telebot.TeleBot(token)
api_request = CountryLayerApi()

# /start
@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn = types.KeyboardButton("Главное меню")
    markup.add(btn)
    bot.send_message(message.from_user.id, "👋 Вас приветствует бот для запросов к сервису https://countrylayer.com/", reply_markup=markup)

# /help
@bot.message_handler(commands=['help'])
def help(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton('/start')
    markup.add(btn1)
    bot.reply_to(message, "Бот справочник может рассказать какие бывают страны а также найти срану по столице или региону.\nВведите /start чтобы перейти к списку запросов.", reply_markup=markup)

# Handler
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    # #Список запросов
    if message.text == 'Главное меню':
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Посмотреть список всех стран мира")
        btn2 = types.KeyboardButton("Поиск по названию страны")
        btn3 = types.KeyboardButton('Поиск по названию столицы')
        btn4 = types.KeyboardButton('Поиск по языку')
        btn5 = types.KeyboardButton('Поиск по валюте')
        btn6 = types.KeyboardButton('Поиск по региону')
        btn7 = types.KeyboardButton('Поиск по региональному блоку')
        btn8 = types.KeyboardButton('Поиск по коду')
        btn9 = types.KeyboardButton('Главное меню')
        markup.add(btn1, btn2, btn3, btn4, btn5, btn6, btn7, btn8, btn9)
        bot.send_message(message.from_user.id, '👀 Выберите способ поиска.', reply_markup=markup)

    # Запрос "Все страны"
    elif message.text == 'Посмотреть список всех стран мира':
        make_response_to_user(message, api_request.request_all())

    # Поиск по названию страны
    elif message.text == 'Поиск по названию страны':
        bot.send_message(message.chat.id, "Введите название страны: ")
        bot.register_next_step_handler(message, bot_request_country_name)

    # Поиск по названию столицы
    elif message.text == 'Поиск по названию столицы':
        bot.send_message(message.chat.id, "Введите название столицы: ")
        bot.register_next_step_handler(message, bot_request_country_capital)

    # Поиск по названию страны
    elif message.text == 'Поиск по языку':
        bot.send_message(message.chat.id, "Введите название столицы: ")
        bot.register_next_step_handler(message, bot_request_country_language)

    # Поиск по валюте
    elif message.text == 'Поиск по валюте':
        bot.send_message(message.chat.id, "Введите название столицы: ")
        bot.register_next_step_handler(message, bot_request_country_currency)

    # Поиск по региону
    elif message.text == 'Поиск по региону':
        bot.send_message(message.chat.id, "Введите название региона: ")
        bot.register_next_step_handler(message, bot_request_country_region)

    # Поиск по региональному блоку
    elif message.text == 'Поиск по региональному блоку':
        bot.send_message(message.chat.id, "Введите название региональному блоку: ")
        bot.register_next_step_handler(message, bot_request_country_regionalbloc)

    # Поиск по коду
    elif message.text == 'Поиск по коду':
        bot.send_message(message.chat.id, "Введите название коду: ")
        bot.register_next_step_handler(message, bot_request_country_calling_code)

    # Anyway
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn = types.KeyboardButton("/help")
        markup.add(btn)
        bot.send_message(message.from_user.id, "Такой команды нет. Введите команду /help.", reply_markup=markup)


# .from_user.id
# Название страны
def bot_request_country_name(message):
    if message.text is not None:
        make_response_to_user(message, api_request.request_country_name(message.text.strip().lower()))

# Название столицы
def bot_request_country_capital(message):
    if message.text is not None:
        make_response_to_user(message, api_request.request_country_capital(message.text.strip().lower()))

# Название языка
def bot_request_country_language(message):
    if message.text is not None:
        make_response_to_user(message, api_request.request_country_language(message.text.strip().lower()))

# Название валюты
def bot_request_country_currency(message):
    if message.text is not None:
        make_response_to_user(message, api_request.request_country_currency(message.text.strip().lower()))

# Название региона
def bot_request_country_region(message):
    if message.text is not None:
        make_response_to_user(message, api_request.request_country_region(message.text.strip().lower()))

# Номер регионального блока
def bot_request_country_regionalbloc(message):
    if message.text is not None:
        make_response_to_user(message, api_request.request_country_regionalbloc(message.text.strip().lower()))

# Номер кода
def bot_request_country_calling_code(message):
    if message.text is not None:
        make_response_to_user(message, api_request.request_country_calling_code(message.text.strip().lower()))

# Response to user
def make_response_to_user(message, response):

    # keyboard
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Главное меню")
    markup.add(btn1)
    markup = markup

    # check errors
    if type(response) is str:
        bot.send_message(message.from_user.id, response)
    else:
        data = json.loads(response.text)

        # record to list
        answer = []
        for i in data:
            tmp = []
            for j in i:
                tmp.append(str(j) + ': ' + str(i[j]))
            answer.append(tmp)

        # text format
        response_to_user = ''
        for i in range(len(answer)):
            response_to_user += '\n'.join(answer[i]) + '\n\n'
        response_to_user = response_to_user.strip()

        # visual features
        list_total_cnt = (len(answer) // 15) + 1 if len(answer) % 15 != 0 else len(answer) // 15

        # preprocessing and sending response message
        if not response_to_user:
            bot.send_message(message.from_user.id, "Данных по запросу не найдено")
        elif len(response_to_user) > 4096:
            for i in range(0, len(answer), 15):
                msg = ""
                for j in range(i, (i + 15)):
                    if j < len(answer):
                        msg += '\n'.join(answer[j]) + '\n\n'
                msg = msg.strip()
                msg = f'List {(i // 15) + 1}/{list_total_cnt}:\n\n' + msg
                if len(msg) <= 4096:
                    bot.send_message(message.chat.id, msg, parse_mode='Markdown')
                else:
                    bot.send_message(message.chat.id, "Message symbol limit overflow", parse_mode='Markdown')
                sleep(1)
        else:
            bot.send_message(message.chat.id, f'List 1/1:\n\n' + response_to_user, parse_mode='Markdown')

    bot.send_message(message.from_user.id, "Для возврата в главное меню нажмите кнопку 'Главное меню'", reply_markup=markup)

# Non-stop polling
bot.polling(none_stop=True, interval=0)

## FastAPI
